In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import regex as re
import misfunciones as mf

In [2]:
#cargamos csv
sharks = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")

In [3]:
sharks.sample(100)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
11912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3877,1961.09.24.a,24-Sep-1961,1961.0,Provoked,TANZANIA,NaN,Near entrance to Dar-es-Salaam Harbour,Fishing,Yusef Mohamed,M,...,NaN,Mombasa Times 9/25/1961,1961.09.24.a-YusefMohamed.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1961.09.24.a,1961.09.24.a,2426.0,NaN,NaN
12629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4632,1944.12.02,02-Dec-1944,1944.0,Provoked,USA,Florida,3 miles north of the inlet at Palm Beach,Fishing for mackerel,"28' sea skiff, occupants: Alan Moree and anoth...",NaN,...,"Tiger shark, 4.5 to 5.5 m [14'9"" to 18'], 2000...","New York Times, 12/3/1944, III, p.2, col.7",1944.12.02-skiff-Moree.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1944.12.02,1944.12.02,1671.0,NaN,NaN
5544,1901.09.23.R,Reported 23-Sep-1901,1901.0,Unprovoked,CYPRUS,Southern Cyprus,Larnaca,Swimming,male,M,...,2 m shark,"Bardanis citing Embros, 9/23/1901",1901.09.23.R-Cyprus.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1901.09.23.R,1901.09.23.R,759.0,NaN,NaN


In [4]:
#eliminamos todas las columnas y filas que todos sus valores sean nulos
sharks.dropna(how="all", inplace=True)
sharks.dropna(how="all",axis = 1, inplace=True)

# Hipótesis
- hipotesis 1 los ataques de tiburones han icrmentado según han ido avanzando los años
- hipótesis 2 los ataquesde tiburones atacan más a embarcaciones que a nadadores
- [hipóstesis 3 Florida es la capital mundial de los ataques de tiburones](https://www.lavanguardia.com/ocio/viajes/20210407/6631447/6-playas-mas-peligrosas-mundo.html)
- [hipótesis 4 ¿Es posible el ataque de tiburón en la costa española?](https://www.mundo-geo.es/naturaleza/es-posible-ataque-tiburon-en-costa-espanola_238643_102.html)

In [5]:
#compruebo qué colmunas tienen datos interesantes para mis hipótesis y borro las que no me hacen ninguna falta.
sharks.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [6]:
columnas_no = ['Case Number','Investigator or Source','pdf', 'href formula',  'href',  'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23'] 
sharks.drop(columnas_no, axis=1, inplace=True)

In [7]:
sharks.columns

Index(['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species '],
      dtype='object')

In [8]:
#ahora que hemos borrado columnas vemos si quedan filas que son nan por completo y borrranos
sharks.dropna(how="all", inplace=True)

In [9]:
#cambio nombre de algunas columnas que tienen espacios o caracteres especiales en el nombre
#"Sex " "Species " "Fatal (Y/N)"
cols = ["Sex ","Species ","Fatal (Y/N)"]
new_names = {"Sex " : "Sex",
                 "Species " : "Species",
                 "Fatal (Y/N)" : "Fatal"}
sharks.rename(columns = new_names, inplace=True)
sharks.columns

Index(['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species'],
      dtype='object')

In [10]:
#segundo repaso a datos para ver si borramos alguna columna más que no me de información necesaria para las hipótesis
sharks.sample(20)
sharks.drop(["Name"],axis=1, inplace=True)

### Limpieza de datos nulos

In [11]:
#miramos cuantos datos nulos hay en cda columna.
sharks.isna().sum()

Date           0
Year           2
Type           4
Country       50
Area         455
Location     540
Activity     544
Sex          565
Age         2831
Injury        28
Fatal        539
Time        3354
Species     2838
dtype: int64

#### rellenamos los nulos en función de los datos que contienen las columnas


In [12]:
sharks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6302 entries, 0 to 6301
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      6302 non-null   object 
 1   Year      6300 non-null   float64
 2   Type      6298 non-null   object 
 3   Country   6252 non-null   object 
 4   Area      5847 non-null   object 
 5   Location  5762 non-null   object 
 6   Activity  5758 non-null   object 
 7   Sex       5737 non-null   object 
 8   Age       3471 non-null   object 
 9   Injury    6274 non-null   object 
 10  Fatal     5763 non-null   object 
 11  Time      2948 non-null   object 
 12  Species   3464 non-null   object 
dtypes: float64(1), object(12)
memory usage: 689.3+ KB


In [13]:
#Year
sharks.Year.unique()
#rellenamos años vacíos con 0
sharks.Year.fillna(0.0,inplace= True)
#convertirmos tipo de dato (float) a int.
sharks.Year = sharks.Year.astype(dtype = "int64")

In [14]:
#type
sharks.Type.unique() #['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable','Sea Disaster', nan, 'Boat', 'Boatomg']
sharks[sharks["Type"] == "Invalid"].sample(20)
#invalid NO nos sirve para los nulos porque es para ataques que no fueron de tiburón
#mirar a ver si podemos eliminar esas filas porque queremos estudiar solo los casos reales de tiburones.
#nos da mucha información si el ataque fue en embarcación o direcgtamente a "nadador"
sharks.Type.fillna("UNKNOWN",inplace= True)

In [15]:
#Country, Area y Location -> nans == UNKNOWN
sharks["Country"].fillna("UNKNOWN", inplace = True)
sharks["Area"].fillna("UNKNOWN", inplace = True)
sharks["Location"].fillna("UNKNOWN", inplace = True)

In [16]:
#activity
list(sharks.Activity.unique())
#unknown
#de esta columna podemos sacar datos sobre si el tiburón atacó a embarcación o persona.
sharks["Activity"].fillna("UNKNOWN", inplace = True)

In [17]:
#sex
sharks.Sex.unique()
#si el sexo de la víctima es desconocido lo rellenamos con "X"
#además queremos que se queden tres datos únicos -> M,F,X
sharks.Sex.fillna("X",inplace = True)
sharks.Sex = sharks.Sex.str.strip() #borramos espacios vacíos antes y después de cada cadena.
sharks.Sex.unique() #['F', 'M', 'X', 'lli', 'N', '.'] > lli, N y . == X
sharks["Sex"].replace('lli',"X",inplace=True)
sharks["Sex"].replace('.',"X",inplace=True)
sharks["Sex"].replace('N',"X",inplace=True)

In [18]:
#Age
sharks.Age.unique()
#rellenamos nans con "UNKNOWN" porque 0 puede confundir la media si luego necesitamos usarla.
sharks.Age.fillna("UNKNOWN",inplace= True)
#quitamos espacios antes y detrás de los strings
sharks.Age = sharks.Age.str.strip()
#convertirmos a número todas las cadenas que sean dígitos
sharks.Age = sharks.Age.apply(mf.entero_if)
#queda pendiente limpiar datos que son cadena y no enteros.
    #posiblidad = nueva columnas con rangos de edad y y poner datos como los strings que ya están.

In [19]:
#Injury > nan = UNKNOWN
sharks.Injury.unique()
sharks["Injury"].fillna("UNKNOWN", inplace = True)

In [20]:
#Fatal
sharks.Fatal.unique
#nulos = UNKNOWN
sharks["Fatal"].fillna("UNKNOWN", inplace = True)
sharks.Fatal = sharks.Fatal.str.strip()
sharks.Fatal.unique() #comprobar registros con FAtal == 'M', '2017', 'y'
sharks[(sharks["Fatal"] == 'M')|(sharks["Fatal"] == "2017")|(sharks["Fatal"] == "y")].Injury.unique
sharks["Fatal"].replace('M',"N",inplace=True)
sharks["Fatal"].replace('y',"Y",inplace=True)
sharks["Fatal"].replace('2017',"N",inplace=True)
sharks.Fatal.unique() #['N', 'Y', 'UNKNOWN']

array(['N', 'Y', 'UNKNOWN'], dtype=object)

In [21]:
#Time
sharks.Time.unique()
#nulos = UNKNOWN
sharks["Time"].fillna("UNKNOWN", inplace = True)
#puede que esta columna no la utlicemos para ningún análisis. No nos interesan los datos tan exactos de la hora del ataque.

In [22]:
#Species
list(sharks.Species.unique())
sharks["Species"].fillna("UNKNOWN", inplace = True)

In [23]:
sharks.isna().sum()

Date        0
Year        0
Type        0
Country     0
Area        0
Location    0
Activity    0
Sex         0
Age         0
Injury      0
Fatal       0
Time        0
Species     0
dtype: int64

#### precesamos el texto de las columnas

##### Date, Year.

In [24]:
sharks.Date.value_counts()
#de la columna Date extraemos solo las fechas
sharks["Year_D"] = sharks["Date"].str.extract(r'(\d{4})')
sharks["Month_D"] = sharks["Date"].str.extract(r'([A-Z][a-z][a-z]-\d{4})')
sharks["Month_D"] = sharks["Month_D"].str.extract(r'([A-Z][a-z][a-z])')
#volvemos a rellenar los nulos que quedan 
#sharks["Date"].fillna("UNKNOWN", inplace = True)

In [25]:
sharks.Year_D.unique() #los registros de esta columna parecen más concretos que la de Year
#rellenamos nans de esta columna y tanbién de la columna Month. elimnamos Year.
sharks.Year_D.fillna("UNKNOWN",inplace= True)
sharks.Month_D.fillna("UNKNOWN",inplace= True)
#convertimos los strings a enteros
sharks.Year_D = sharks.Year_D.apply(mf.entero_if)


In [26]:
# miramos los registros cuyo tipo es Invalid para ver si nos sirven o no... 
sharks[(sharks.Type == "Invalid")].sample(50)
# muchos de los registros confirman que no son ataques de tiburón, otros son ataques de tiburón muy dudosos. 
# nos deshacemos de los registros que tengan typo = "INVALID"

sharks = sharks[(sharks.Type != "Invalid")]


In [27]:
#como tenemos unos datos más coherentes con month_D y Year_D borramos Date y year
sharks.drop(["Date","Year"],axis=1, inplace=True)

##### Type

In [28]:
sharks.Type.unique()

array(['Boating', 'Unprovoked', 'Provoked', 'Questionable',
       'Sea Disaster', 'UNKNOWN', 'Boat', 'Boatomg'], dtype=object)

In [29]:
sharks[(sharks["Type"] == "Boating")|(sharks["Type"] == "Boatomg")|(sharks["Type"] == "Boat")].sample(50) #son el mismo tipo
#cambiamos todos a "Boat"
sharks["Type"] = sharks["Type"].str.replace((".*[Bb](OAT|oat).*"),"Boat", regex=True)
sharks.Type.unique()

array(['Boat', 'Unprovoked', 'Provoked', 'Questionable', 'Sea Disaster',
       'UNKNOWN'], dtype=object)

In [30]:
#sharks[["Type","Activity","Injury"]][(sharks.Type != "Boat")&(sharks.Activity.str.contains(".*[Ff](ishin|ISHIN).*",regex = True))].sample(50)
#sharks[["Type","Activity","Injury"]][(sharks.Type != "Boat")&(sharks.Injury.str.contains(".*[Oo](c+upan|C+UPAN).*",regex = True))]
#sharks[["Type","Activity","Injury"]][((sharks.Type != "Boat")&(sharks.Injury.str.contains(".*[Oo](c+upan|C+UPAN).*",regex = True)))|(sharks.Injury.str.contains(".*[Oo](c+upan|C+UPAN).*",regex = False))]

In [31]:
#comprobamos los datos de las columnas Type, Activity e Injury que son las que más datos nos dan de si el tiburón ataca botes
sharks[["Type","Activity","Injury"]]
#ponemos filtro a Type para que sea diferente de Boat y así ver si quedan datos que aporten ataques a embarcaciones
sharks[["Type","Activity","Injury"]][(sharks.Type != "Boat")].sample(50)
#algunos registros dicen en Injury = "No injury to ocuppant(s)" por lo que podemos deducir que se atacó embarcación:
sharks[["Type","Activity","Injury"]][(sharks.Type != "Boat")&(sharks.Injury.str.contains(".*[Oo](c+upan|C+UPAN).*",regex = True))]
#algunos registros de Activity parece ser que ocurrieron en embarcaciones en las que se estaba pescando -> Ataque embarcación.
sharks[["Type","Activity","Injury"]][(sharks.Type != "Boat")&(sharks.Activity.str.contains("(\s|^)[Ff](ishing|ISHING).*",regex = True))].sample(10)

<ipython-input-31-14e1ba21cf7c>:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  sharks[["Type","Activity","Injury"]][(sharks.Type != "Boat")&(sharks.Injury.str.contains(".*[Oo](c+upan|C+UPAN).*",regex = True))]
<ipython-input-31-14e1ba21cf7c>:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  sharks[["Type","Activity","Injury"]][(sharks.Type != "Boat")&(sharks.Activity.str.contains("(\s|^)[Ff](ishing|ISHING).*",regex = True))].sample(10)


,Type,Activity,Injury
3825,Provoked,Fishing,"No injury to occupants, hooked shark cracked h..."
1529,Provoked,Fishing for sharks,Lacerations to hands & right leg when he tried...
290,Unprovoked,Fishing,Left hand bitten
5020,Unprovoked,"Fishing, boat capsized",FATAL
4547,Provoked,"Fishing, shark caught in his net","8"" x 4"" left antecubital fossa of muscle & ski..."
337,Unprovoked,Fishing,"Arms severely injured, surgically amputated"
5021,Unprovoked,Fishing with a cast net,"FATAL, left thigh severely bitten"
5340,Provoked,Fishing,PROVOKED INCIDENT Lacerations to right hand by...
5395,Unprovoked,Fishing,FATAL
5217,Unprovoked,Fishing,FATAL


In [32]:
#reemplazamos datos en Activity para hacerlos más concisos.
list(sharks.Activity.unique())
sharks.Activity = sharks.Activity.str.strip()

In [33]:
len(list(sharks.Activity.unique()))

1392

In [34]:
list(sharks.Activity.unique())
#("(\s|^)[Ff](ishing|ISHING).*",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ss](URF|urf).*","Surf and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Bb](oog|OOG).*","Surf and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Kk](ITE|ite).*","Surf and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Bb](ody|ODY).*[Bb](OARD|oard).*","Surf and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*([Dd](ivin|IVIN)|[Dd](ive|IVE)).*","Diving and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ss](wim+|WIM+).*","Swimming and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Jj](ump|UMP).*","Swimming and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Pp](ad+|AD+).*","Paddling and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ss](NORK|nork).*","Snorkeling and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ff](E+DIN|e+din).*","Feeding animals and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Sp](EAR|ear).*","Using spears",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Kk](ayak|AYAK).*","Kayaking and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Cc](anoe|ANOE).*","Canoeing and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Bb](ATH|ath).*","Swimming and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ff](LOAT|loat).*","Swimming and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ss](ki|KI).*","Skiing and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Pp](LAY|lay).*","Playing and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Kk](ILL|ill).*","Fishing and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*([Cc](apsi|APSI)|[Ss](ink|INK)).*","Capsized, sinked",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ww](reck|RECK).*","Shipwreck",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Dd](eep|EEP).*","Diving and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ff](ilm|ILM).*","Filming or photographing",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Pp](HOT|hot).*","Filming or photographing",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ss](elfi|ELFI).*","Filming or photographing",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Rr](OW|ow).*","Rowing and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*([Ss](TAND|tand)|[Ss](itt|ITT)).*","Standing/Sitting",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ss](ail|AIL).*","sailing and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ww](ash|ASH).*","washing",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ww](adin|ADIN).*","wading",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Dd](isast|ISAST).*","Air/Sea Disaster",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Aa](irc|IRC).*","Air/Sea Disaster",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Rr](ESCU|escu).*","Rescuing and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Aa](DRIFT|drift).*","Adrift",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Oo](VERBO|verbo).*","Fell/swept overboard",regex = True)
#overboard 


sharks["Activity"] = sharks["Activity"].str.replace(".*[Ff](ish|ISH).*","Fishing and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ff](inn|INN).*","Fishing and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Hh](UNT|unt).*","Fishing and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ss](hrim|HRIM).*","Fishing and similars",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ll](OBST|obst).*","Fishing and similars",regex = True)
#'Lobstering'

In [35]:
list(sharks.Activity.unique())

['Paddling and similars',
 'Standing/Sitting',
 'Surf and similars',
 'Diving and similars',
 'Swimming and similars',
 'Fishing and similars',
 'Walking',
 'Feeding animals and similars',
 'wading',
 'Kayaking and similars',
 'Snorkeling and similars',
 'UNKNOWN',
 'Using spears',
 'Capsized, sinked',
 'Canoeing and similars',
 'SUP',
 'Skiing and similars',
 'Touching a shark',
 'Attempting to lasso a shark',
 'Kakaying',
 'washing',
 'Filming or photographing',
 'Tagging sharks',
 'SUP Foil boarding',
 'Teasing a shark',
 'Air/Sea Disaster',
 'Playing and similars',
 'Rescuing and similars',
 'Rowing and similars',
 'Petting a shark',
 'Kneeling in the water',
 'Fell into the water',
 'Shark watching',
 'sailing and similars',
 'Casting a net',
 'Wrangling a shark',
 'Attempting to free the shark',
 'Wakeboarding',
 'Attempting to fix motor',
 'Measuring sharks',
 'Yacht race',
 'Treading water',
 "Accidentally stood on hooked shark's tail before attempting to gut it",
 'Attempting 

In [36]:
len(list(sharks.Activity.unique()))

325

In [37]:
#hacer una nueva columna booleana "boat" 1 = el tibruón ataca a embarcación. 0 = tiburón ataca persona en agua.
#incluir datos que de Type == Boat
sharks[(sharks["Type"] == "Boat")]
#también incluir datos que Type != de Boat pero en activity estuvieran pescando y en injury "No injury to occupants"
#sharks[["Type","Activity","Injury"]][(sharks.Type != "Boat")&(sharks.Activity.str.contains("(\s|^)[Ff](ishing|ISHING).*",regex = True))]
sharks[(sharks["Type"] == "Boat")|((sharks.Type != "Boat")&(sharks.Activity.str.contains("(\s|^)[Ff](ishing|ISHING).*",regex = True)))]

<ipython-input-37-2504f9a387e7>:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  sharks[(sharks["Type"] == "Boat")|((sharks.Type != "Boat")&(sharks.Activity.str.contains("(\s|^)[Ff](ishing|ISHING).*",regex = True)))]


,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal,Time,Species,Year_D,Month_D
0,Boat,USA,California,"Oceanside, San Diego County",Paddling and similars,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,2018,Jun
7,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing and similars,M,52,Minor injury to foot. PROVOKED INCIDENT,N,UNKNOWN,"Lemon shark, 3'",2018,May
23,Unprovoked,MALDIVES,Alifu Alifu Atoll,Madoogali,Fishing and similars,M,32,5-inch cut to hand,N,21h50,Tiger shark,2018,Apr
85,UNKNOWN,SAMOA,Upolu Island,Nofoalii,Fishing and similars,M,UNKNOWN,Injuries to hands and legs,N,Night,UNKNOWN,2017,Sep
86,Boat,AUSTRALIA,Westerm Australia,Esperance,Fishing and similars,X,UNKNOWN,"sharks rammed boats, no injury to occupants",N,UNKNOWN,"White shark, 3.5m",2017,Sep
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6283,Unprovoked,USA,Florida,"Gadsden Point, Tampa Bay",Fishing and similars,M,UNKNOWN,2-inch lacerations,N,UNKNOWN,UNKNOWN,1921,UNKNOWN
6289,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,Durban,Fishing and similars,M,UNKNOWN,"FATAL, body not recovered but shark was caught...",Y,UNKNOWN,UNKNOWN,1913,Jul
6293,Unprovoked,AUSTRALIA,UNKNOWN,UNKNOWN,Fishing and similars,M,UNKNOWN,"FATAL, knocked overboard by tail of shark & ca...",Y,UNKNOWN,Blue pointer,1906,UNKNOWN
6294,Unprovoked,AUSTRALIA,UNKNOWN,UNKNOWN,Fishing and similars,M,UNKNOWN,FATAL,Y,UNKNOWN,Blue pointer,1906,UNKNOWN


In [38]:
#me gustaría hacer una columna que solo pusiera "boat" e incluir un 1 por toods lso type = Boat
    #también incluir todos los #activity == fishing and similars
#para ver los tipos de 

## species

In [39]:
len(list(sharks.Species.unique())) #1493
sharks.Species = sharks.Species.str.strip()

In [40]:
sharks["Species"] = sharks["Species"].str.replace(".*[Uu](NKNOW|nknow).*","UNKNOWN",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Ww](HITE|hite).*","White",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Tt](iger|IGER).*","Tiger",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Ll](emon|EMON).*","Lemon",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Bb](UL|ul).*","Bull",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Nn](URSE|urse).*","Nurse",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Ss](pin|PIN).*","Spinner",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Hh](ammer|AMMER).*","Hammerhead",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Ll](UE|ue).*","Blue",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Mm](ako|AKO).*","Mako",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Bb](lacktip|LACKTIP).*","Blacktip",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Ww](obbegong|OBBEGONG).*","Wobbegong",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Rr](eef|EEF).*","Reef",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Bb](ronze|RONZE).[Ww](haler|HALER).*","Bronze whaler",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Rr](agged|AGGED).*","Raggedtooth",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Ss](and|and).*","Sandbar",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Gg](R(A|E)Y|r(a|e)y).*","Grey",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Ww](HALE|hale)\s.*","Whale",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Zz](ambe|AMBE).*","Zambesi",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Bb](LACK|lack).[T|t].*","Blacktip",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*\d.*","UNKNOWN",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Uu](niden|NIDEN).*","UNKNOWN",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Ss](mall|MALL)\s.*","UNKNOWN",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Ll](ittle|ITTLE)\s.*","UNKNOWN",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Jj](uvenile|UVENILE).*","UNKNOWN",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Yy](oung|OUNG).*","UNKNOWN",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Ll](ARGE|arge).*","UNKNOWN",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Uu](NKNOW|nknow).*","UNKNOWN",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Rr](ecover|ECOVER).*","UNKNOWN",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Ss](chool|CHOOL).*","School",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Pp](ack|ACK)\s.*","School",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Nn](UMBER|umber).*","School",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Ss](EVERAL|everal).*","School",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Ss](hark|HARK).*","Other",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Cc].\s.*","Other",regex = True)
sharks["Species"] = sharks["Species"].str.replace(".*[Ss](hovel|HOVEL).*","Other",regex = True)
#sharks["Species"] = sharks["Species"].replace("''","UNKNOWN",inplace = True)

#Wobbegong 

In [41]:
prueba = pd.DataFrame(sharks.Species.value_counts())
prueba

,Species
UNKNOWN,3869
White,657
Tiger,281
Bull,174
Nurse,97
Blacktip,78
Other,69
Bronze whaler,63
Blue,55
Mako,53


In [42]:
list(sharks.Species.unique())

['White',
 'UNKNOWN',
 'Tiger',
 'Lemon',
 'Bull',
 'Reef',
 'Nurse',
 'Other',
 'Questionable',
 'Wobbegong',
 'Blacktip',
 'Blue',
 'Spinner',
 'Mako',
 'Bronze whaler',
 'Hammerhead',
 'Raggedtooth',
 'Sandbar',
 'Zambesi',
 'Spurdog',
 'Whale',
 '',
 'Grey',
 'School']

In [43]:
len(list(sharks.Species.unique()))

24

In [44]:
sharks.sample(10)

,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal,Time,Species,Year_D,Month_D
1746,Unprovoked,USA,South Carolina,"Isle of Palms, Charleston County",Standing/Sitting,M,15,Ankle lacerated,N,Afternoon,UNKNOWN,2003,Sep
5035,Unprovoked,HONDURAS,Black River,UNKNOWN,Swimming and similars,M,UNKNOWN,FATAL,Y,UNKNOWN,UNKNOWN,1930,Sep
646,Unprovoked,BAHAMAS,Abaco Islands,Scotland Cay,Using spears,M,40,Leg bitten,N,16h05,UNKNOWN,2013,Jul
3426,Provoked,ENGLAND,UNKNOWN,UNKNOWN,UNKNOWN,M,UNKNOWN,Arm lacerated. Recorded as PROVOKED INCIDENT,N,UNKNOWN,UNKNOWN,1969,Jul
4224,Unprovoked,GABON,Estuaire Province,Owendo,Swimming and similars,M,UNKNOWN,FATAL,Y,UNKNOWN,UNKNOWN,1956,Dec
4489,Unprovoked,INDONESIA,Jakarta Harbour,On rock near Yacht Club,Bending over,M,7 or 8,FATAL,Y,1500,Tiger,1950,UNKNOWN
3048,Unprovoked,SOUTH AFRICA,Eastern Cape Province,Nahoon,Paddling and similars,M,25,"No injury, paddleski bitten",N,17h15,UNKNOWN,1980,UNKNOWN
2609,Unprovoked,AUSTRALIA,Western Australia,Scarborough,Surf and similars,M,UNKNOWN,"No injury, shark brushed ski",N,UNKNOWN,UNKNOWN,1991,Jan
5824,Unprovoked,AUSTRALIA,New South Wales,Cooranbong,Swimming and similars,F,13,Lacerations to leg,N,UNKNOWN,UNKNOWN,1880,Jan
851,Unprovoked,USA,Oregon,"Newport, Lincoln County",Surf and similars,M,41,"No injury, shark bit surfboard",N,UNKNOWN,White,2011,Oct


Hacemos que los datos sean más concretos en la columnas.

In [45]:
#creamos un sub dataframe con solo las colmunas que realmente necesitamos.
sharky = sharks[["Type","Country","Area","Activity","Sex","Age","Fatal","Species","Year_D","Month_D"]]

In [46]:
#exportarmos sharky
sharky.to_csv("data/sharks.csv")

In [47]:
#comprobamos que los registros que tienen como unknown area, location y Country
#sharks[(sharks.Country == "UNKNOWN")|(sharks.Country == "UNKNOWN")|(sharks.Country == "UNKNOWN")].sample(20)

In [48]:
sharks

,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal,Time,Species,Year_D,Month_D
0,Boat,USA,California,"Oceanside, San Diego County",Paddling and similars,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White,2018,Jun
1,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing/Sitting,F,11,Minor injury to left thigh,N,14h00 -15h00,UNKNOWN,2018,Jun
3,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surf and similars,M,UNKNOWN,Minor injury to lower leg,N,UNKNOWN,UNKNOWN,2018,Jun
4,Provoked,MEXICO,Colima,La Ticla,Diving and similars,M,UNKNOWN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,UNKNOWN,Tiger,2018,Jun
5,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Surf and similars,M,UNKNOWN,"No injury, board bitten",N,UNKNOWN,UNKNOWN,2018,Jun
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving and similars,M,UNKNOWN,FATAL,Y,UNKNOWN,UNKNOWN,1903,UNKNOWN
6298,Unprovoked,AUSTRALIA,Western Australia,UNKNOWN,Diving and similars,M,UNKNOWN,FATAL,Y,UNKNOWN,UNKNOWN,1903,UNKNOWN
6299,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming and similars,M,UNKNOWN,FATAL,Y,UNKNOWN,UNKNOWN,1900,UNKNOWN
6300,Unprovoked,PANAMA,UNKNOWN,"Panama Bay 8ºN, 79ºW",UNKNOWN,M,UNKNOWN,FATAL,Y,UNKNOWN,UNKNOWN,1883,UNKNOWN
